### The commands which is important when we are using Postgres/PostGIS

## Information about Table in DB

In [ ]:
SELECT f_geometry_column, coord_dimension,
      srid, type 
FROM geometry_columns
WHERE f_table_name = 'tes_table';

## Spatial join  (Two shapefiles)

In [ ]:
select g.brightness,g.confidence,c.iso2,
       g.the_geom into chp01.qtest
from global_24h as g
join countries as c
on st_contains(c.the_geom,st_transform(g.the_geom,4326));

## Re-Projection

In [ ]:
ALTER TABLE chp01.global_24h
ALTER COLUMN the_geom
SET DATA TYPE geometry(Point, 3857)
USING ST_Transform(the_geom, 3857);

## Saving shapefile

In [ ]:
pgsql2shp -f subregions.shp -h localhost -u behzad -P gis postgis "select subregion, st_union(the_geom) as the_geom, sum(pop2005) as pop2005 from chp01.countries group by subregion;"

## Importing Shapfile into the DB

In [ ]:
shp2pgsql -I -d -s 4326 -W LATIN1 -g the_geom countries.shp chp01.countries > countries.sql

In [ ]:
psql -U behzad -d postgis -f countries.sql

## Importing Raster into DB

In [ ]:
raster2pgsql -I -C -F -t 100x100 -s 4326 tmax1.bil chp01.tmax01 > tmax01.sql

In [ ]:
psql -d postgis -U behzad -f tmax01.sql

## Show the Raster Info.

In [ ]:
# Info about all rasters in the DB
SELECT * FROM raster_columns;

In [ ]:
SELECT r_raster_column, srid,
      ROUND(scale_x::numeric, 2) AS scale_x,
      ROUND(scale_y::numeric, 2) AS scale_y, blocksize_x,
      blocksize_y, num_bands, pixel_types, nodata_values, out_db
      FROM raster_columns where r_table_schema='chp01'
      AND r_table_name ='tmax2012';

In [ ]:
# some raster statistics using the ST_MetaData function 
# (Note that there is different metadata for each raster record loaded in the table.)

SELECT rid, (foo.md).*
      FROM (SELECT rid, ST_MetaData(rast) As md
      FROM chp01.tmax2012) As foo;

## Show the coordinate of the polygon

In [ ]:
select country_name, st_astext(the_geom) as wkt_geom
from chp01.africa_countries 
order by country_name 
limit DESC 3

## Errors after creation table (Primary key & Geometry_columns)

In [ ]:
## Geometry : http://movingspatial.blogspot.com/2013/01/postgis-15-batch-populate-geometry.html

SELECT Populate_Geometry_Columns('schema.table'::regclass);

In [ ]:
## Primary key

ALTER TABLE t1 ADD PRIMARY KEY(id);

ALTER TABLE qlayer ADD ID SERIAL PRIMARY KEY;

# Make a point and set coordinate system

In [ ]:
ST_SetSRID

In [ ]:
ST_MakePoint